# DL Survival - Ventilation Outcomes
 Updated 21/11/21

In [ ]:
import pandas as pd
import numpy as np
import math
import statistics
from datetime import datetime
import datetime as dt
from datetime import timedelta
import json
import miceforest as mf
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

pd.set_option('display.max_columns', None)

## 1. Data cleaning

- Import MIMIC III data
- Review column unique values, assign correct data types
- Impute missing values


### 1.1: Importing data

In [ ]:
df = pd.read_csv('mimic_combined.csv')

In [ ]:
pd.set_option('display.max_columns', None)
print(df.shape)
df.head(5)

#### 1.1.1: Column lists

In [ ]:
#view and reorder columns
cols = list(df.columns)
new_cols = ['Unnamed: 0','hadm_id','subject_id','gender','ethnicity','marital_status','insurance','language','aortic','mit','tricuspid',
            'pulmonary','cabg','temp','bg_temp','hr','spo2','rr','sbp','dbp','meanbp','weight','height','cardiac_index','pt','ptt',
            'inr','inr_1','fibrinogen','hb','hematocrit','plts','wcc','lymphocytes','neutrophils','alp','ast','alt','ggt',
            'bilirubin_indirect','bilirubin_direct','bilirubin_total','chloride','magnesium','potassium','crp','bleed_time','albumin',
            'creatinine','free_calcium','sodium','bicarb','bun','hba1c','glucose','lactate','po2','pco2','baseexcess','ph','aado2',
            'fio2','ffp','insulin','cryo','prbc','infection','ventrate','tidalvol','vent_array','reintubation','liver_severe','liver_mild',
            'rheum','cvd','aids','ckd','copd','arrhythmia','pud','smoking','pvd','paraplegia','ccf','met_ca','t2dm','t1dm','malig','mi',
            'dementia','first_careunit','last_careunit','admission_location','admission_type','hospital_expire_flag','admittime',
            'dischtime','intime','outtime','ext_time','reint_time','los','icustay_seq','deathtime','plt','diab_un','diab_cc',
            'dtoutput','specimen','dod']

ptinfo=['Unnamed: 0','hadm_id','subject_id']

demographics=['gender','ethnicity','marital_status','insurance','language']

proceduretype=['aortic','mit','tricuspid','pulmonary','cabg']

vitals=['temp','bg_temp','hr','spo2','rr','sbp','dbp','meanbp','weight','height','cardiac_index']

labs=['pt','ptt','inr','inr_1','fibrinogen','hb','hematocrit','plts','wcc','lymphocytes','neutrophils','alp','ast','alt','ggt',
'bilirubin_indirect','bilirubin_direct','bilirubin_total','chloride','magnesium','potassium','crp','bleed_time',
'albumin','creatinine','free_calcium','sodium','bicarb','bun','hba1c','glucose','lactate']

bloodgases=['po2','pco2','baseexcess','ph','aado2','fio2']

products=['ffp','insulin','cryo','prbc','infection']

ventilation=['ventrate','tidalvol','vent_array','reintubation']

comorbidities=['liver_severe','liver_mild','rheum','cvd','aids','ckd','copd','arrhythmia','pud','smoking','pvd',
'paraplegia','ccf','met_ca','t2dm','t1dm','malig','mi','dementia']

adm_cat=['first_careunit','last_careunit','admission_location','admission_type','hospital_expire_flag']

adm_num=['admittime','dischtime','intime','outtime','ext_time','reint_time','los','icustay_seq','deathtime']

others=['plt','diab_un','diab_cc','dtoutput','specimen','dod']

timeseries=[*vitals,*labs,*bloodgases,*products,*ventilation,'plt','dtoutput']
timeseries = [i for i in timeseries if i not in ('weight','height','reintubation', 'infection', 'vent_array')]
    
timeseries_valuenames = {'cardiac_index':'ci',
                         'plts':'bloodproduct',
                         'ffp':'bloodproduct',
                         'insulin':'amount',
                         'cryo':'bloodproduct',
                         'prbc':'bloodproduct',
                         'dtoutput':'output'}

In [ ]:
df = df[new_cols]
df.head(10)

### 1.2: Cleaning data types

#### 1.2.0: NaN assignment

In [ ]:
df = df.replace('NaT',np.datetime64('NaT'))
df = df.replace(['[]','NaN',np.datetime64('NaT')],np.NaN)

#### 1.2.1: Datetime columns

In [ ]:
# set column types as datetime
time_cols = ['admittime','dischtime','intime','outtime','reint_time','ext_time','deathtime']
for col in time_cols:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')

#dod
df['dod'] = pd.to_datetime(df['dod'], format='%Y-%m-%d')

In [ ]:
## CHECK FOR ROWS WHERE DEATHTIME < INTIME OR ADMITTIME

In [ ]:
df[time_cols].dtypes

#### 1.2.2: Demographics

In [ ]:
for x in demographics:
    print(x,': ',df[x].unique())

In [ ]:
#ethnicity
df.replace({'ethnicity':
                {'unknown': np.NaN,'UNKNOWN':np.NaN,'UNABLE TO OBTAIN':np.NaN,
                'OTHER':'other','WHITE':'white','BLACK/AFRICAN AMERICAN':'black','ASIAN':'asian',
                'HISPANIC/LATINO':'hispanic','AMERICAN INDIAN/ALASKA NATIVE':'native'
                }
            }, 
            inplace=True)
print(df['ethnicity'].unique())

In [ ]:
#marital_status
df.replace({'marital_status':
                {'UNKNOWN (DEFAULT)': np.NaN
                }
            }, 
            inplace=True)
print(df['marital_status'].unique())

In [ ]:
#language
df.replace({'language':
                {'ENGLISH':'ENGL','?':np.NaN
                }
            }, 
            inplace=True)
print(df['marital_status'].unique())

#### 1.2.3: ✔Procedure type

In [ ]:
for x in proceduretype:
    print(x,': ',df[x].unique())

#### 1.2.4: **Vitals / Blood Gases / Products + infection / Ventilation


In [ ]:
# wait for Jahan/others
# ventrate seems to be empty

#### 1.2.5: ✔Comorbidities

In [ ]:
for x in comorbidities:
    print(x,': ',df[x].unique())

#### 1.2.6: Admissions (categorical)

In [ ]:
for x in adm_cat:
    print(x,': ',df[x].unique())

In [ ]:
#first_careunit
df.replace({'first_careunit':
                {'Cardiac Vascular Intensive Care Unit (CVICU)':'CVICU',
                'Coronary Care Unit (CCU)':'CCU',
                'Medical Intensive Care Unit (MICU)':'MICU',
                'Surgical Intensive Care Unit (SICU)':'SICU',
                'Neuro Intermediate':'Neuro Inter',
                'Medical/Surgical Intensive Care Unit (MICU/SICU)':'MICU/SICU',
                'Trauma SICU (TSICU)':'TSICU',
                'Neuro Surgical Intensive Care Unit (Neuro SICU)':'Neuro SICU'
                }
            }, 
            inplace=True)
print(df['first_careunit'].unique())

In [ ]:
#last_careunit
df.replace({'last_careunit':
                {'Cardiac Vascular Intensive Care Unit (CVICU)':'CVICU',
                'Coronary Care Unit (CCU)':'CCU',
                'Medical Intensive Care Unit (MICU)':'MICU',
                'Surgical Intensive Care Unit (SICU)':'SICU',
                'Neuro Intermediate':'Neuro Inter',
                'Medical/Surgical Intensive Care Unit (MICU/SICU)':'MICU/SICU',
                'Trauma SICU (TSICU)':'TSICU',
                'Neuro Surgical Intensive Care Unit (Neuro SICU)':'Neuro SICU'
                }
            }, 
            inplace=True)
print(df['last_careunit'].unique())

In [ ]:
#admission_location
df.replace({'admission_location':
                {'TRANSFER FROM HOSP/EXTRAM':'TRANSFER FROM HOSPITAL',
                'PHYS REFERRAL/NORMAL DELI':'PHYSICIAN REFERRAL',
                'TRANSFER FROM SKILLED NUR':'TRANSFER FROM SKILLED NURSING FACILITY',
                'INFORMATION NOT AVAILABLE':np.NaN,
                'CLINIC REFERRAL':'CLINIC REFERRAL/PREMATURE',
                'EMERGENCY ROOM ADMIT':'EMERGENCY ROOM',
                }
            }, 
            inplace=True)
print(df['admission_location'].unique())

#### 1.2.7: Others

In [ ]:
# for x in others:
#     print(x,': ',df[x].unique())

### Parsing time series data

In [ ]:
df['vent_array'][14]

In [ ]:
def va_parser(row, output=6):
    """
    Takes row from `df` returns a list of starttime, endtime, vent duration 
    for first and (if applicable) second intubations

    Parameters
    ----------
    row : row in df
    output_ : select which output you want (use list index below) - e.g. args=[6] for all output when using df.apply()

    Returns
    -------
    single list variable containing  
        [0] int_time1: first intubation starttime
        [1] ext_time1: first intubation endtime
        [2] duration1: first intubation duration
        [3] int_time2: second intubation starttime
        [4] ext_time2: second intubation endtime 
        [5] duration2: second intubation duration
        [6] all

    """
    int_time1=np.NaN
    ext_time1=np.NaN
    duration1=np.NaN
    int_time2=np.NaN
    ext_time2=np.NaN
    duration2=np.NaN
    value = row['vent_array']
    list=[]
    '''a = value
    print(value)'''
    if value == np.NaN or pd.isna(value):
        return np.NaN
    a = value.replace("'",'"')
    a = a.replace('\n ...\n',',').replace('\n', ',').replace('...', '')
    a = a.replace('datetime.','"dt.')
    a = a.replace('),', ')",')
    a = json.loads(a)
    b = [(i['starttime'], i['endtime'], i['duration_hours']) for i in a]
    int_time1=dt.datetime.strptime(b[0][0],'dt.datetime(%Y, %m, %d, %H, %M)')
    ext_time1=dt.datetime.strptime(b[0][1],'dt.datetime(%Y, %m, %d, %H, %M)')
    duration1=b[0][2]
    
    if output==0:
        return int_time1
    if output==1:
        return ext_time1
    if output==2:
        return duration1

    if len(b)>=2:
        int_time2=dt.datetime.strptime(b[1][0],'dt.datetime(%Y, %m, %d, %H, %M)')
        ext_time2=dt.datetime.strptime(b[1][1],'dt.datetime(%Y, %m, %d, %H, %M)')
        duration2=b[1][2]
    if output==3:
        return int_time2
    if output==4:
        return ext_time2
    if output==5:
        return duration2
    if output==6:
        return int_time1, ext_time1, duration1, int_time2, ext_time2, duration2

In [ ]:
df['int_time1']=df.apply(va_parser, args=[0], axis=1)
df['ext_time1']=df.apply(va_parser, args=[1], axis=1)
df['duration1']=df.apply(va_parser, args=[2], axis=1)
df['int_time2']=df.apply(va_parser, args=[3], axis=1)
df['ext_time2']=df.apply(va_parser, args=[4], axis=1)
df['duration2']=df.apply(va_parser, args=[5], axis=1)
df.head()

In [ ]:
def infection_parser(value, timelimit):
    if value == np.NaN:
        return np.NaN
    else:
        a = value
        a = a.replace('\n ','')
        a = a.replace('[','')
        a = a.replace(']','')
        a = a.replace("{'charttime': datetime.datetime",'')
        split = a.split('}')

In [ ]:
# df_infection

In [ ]:
def ts_parser2(value, timeDelta=None, timeLimits=None, valuename='value'):
    # timeDelta is timedelta in hours from earliest entry
    # timeLimits = (startTime, endTime)
    # if both timeDelta and timeLimits are provided, timeDelta overrules.
    # if both are None, then all timepoints are accepted
    if value == np.NaN or pd.isna(value):
        return np.NaN, np.NaN, np.NaN
    a = value.replace("'", '"')
    a = a.replace('\n ...\n',',').replace('\n', ',').replace('...', '')
    a = a.replace('datetime.', '"dt.')
    a = a.replace(f'), "{valuename}"', f')", "{valuename}"')
    a = a.replace('"unit": None', '"unit": "None"')
    a = a.replace('starttime', 'charttime')
    a = json.loads(a)
    b = [(eval(i['charttime']), i[valuename]) for i in a]
    
    if timeDelta:
        startTime = min(b, key=lambda x:x[0])[0]
        inc_b = [i[1] for i in b if i[0] <= startTime + dt.timedelta(hours=timeDelta)]
    else:
        if timeLimits:
            inc_b = [i[1] for i in b if i[0] >= timeLimits[0] and i[0] <= timeLimits[1]]
        else:
            inc_b = [i[1] for i in b]
    if len(inc_b) == 0:
        return np.NaN, np.NaN, np.NaN
    
    return sum(inc_b) / len(inc_b), max(inc_b), min(inc_b)

# test_x = df[timeseries].iloc[0,0]
# print(ts_parser(test_x,12))
# print(ts_parser2(test_x, timeDelta=12))
# print()
# test_y = df['bg_temp'][9]
# print(test_y)
# print('Parser1: ', ts_parser(test_y, 36))
# print('Parser2: ', ts_parser2(test_y, timeDelta=36))

In [ ]:
df

In [ ]:
# delete all rows where int_time1 OR ext_time1 are missing
df = df[~(pd.isnull(df['int_time1']) | pd.isnull(df['ext_time1']))]
df = df.reset_index()
df = df.drop(axis=1, columns=['index', 'Unnamed: 0'], inplace=False)
df

for column in timeseries:
    x = timeseries_valuenames[column] if column in timeseries_valuenames else "value"
    meanList = []
    maxList = []
    minList = []
    for i in df[column]:
        y = ts_parser2(i, timeDelta=36, valuename=x)
        meanList.append(y[0])
        maxList.append(y[1])
        minList.append(y[2])
    df[column+'_mean']=meanList
    df[column+'_max']=maxList
    df[column+'_min']=minList

In [ ]:
df

In [ ]:
#set limit and get list of variables missing above limit in `missing_cols`
missing_limit = 50
missing_cols = missing_data.loc[missing_data['% Missing Values']>missing_limit].index.tolist()
print(missing_cols)
missing_data = missing_data.loc[missing_data['% Missing Values']>missing_limit]
missing_data = missing_data.sort_values(by=['% Missing Values'])
pd.set_option('display.max_rows', None)
missing_data

### 1.3: Handling missing data

#### 1.3.0 Assessing for missing data

In [ ]:
pd.reset_option('display.max_rows')

In [ ]:
# missing_data.loc[time_cols,:]

In [ ]:
dfDroppedMissing = df[[i for i in df.columns if i not in missing_data.index]]
print(list(dfDroppedMissing.columns))

In [ ]:
#set limit and get list of variables missing above limit in `missing_cols`
missing_limit = 20
missing_cols = missing_data.loc[missing_data['% Missing Values']>missing_limit].index.tolist()
print(missing_cols)
missing_data.loc[missing_data['% Missing Values']>missing_limit]

In [ ]:
# option 3: multiple imputation

# x = missing_data.loc[missing_data['% Missing Values']> 0]
# x.loc[[i for i in x.index if i not in time_cols],:]

#### 1.3.1 Creating summary fields for time-series data

In [ ]:
# checking that ts_parser2() works for the timeseries columns

# for j in timeseries:
#     for i in range(len(df[j])):
#         try:
#             if j in timeseries_valuenames:
#                 ts_parser2(df[j][i], timeDelta=36, valuename=timeseries_valuenames[j])
#             else:
#                 ts_parser2(df[j][i], timeDelta=36)
#         except
#             print(j, i)
#             break
#     print(j, 'Fine')

#### 1.3.2 Beginning imputation

In [ ]:
dfDroppedMissing['int_time1']

In [ ]:
dfForImpute = pd.DataFrame([0 for i in range(dfDroppedMissing.shape[0])])

# generating timeseries summary values
for column in timeseries:
    if column not in dfDroppedMissing.columns:
        continue
    x = timeseries_valuenames[column] if column in timeseries_valuenames else "value"
    meanList = []
    maxList = []
    minList = []
    for i in range(len(dfDroppedMissing[column])):
        inTime = dfDroppedMissing['int_time1'][i].to_pydatetime()
        y = ts_parser2(dfDroppedMissing[column][i], timeLimits=(inTime, inTime+dt.timedelta(hours=12)), valuename=x)
        meanList.append(y[0])
        maxList.append(y[1])
        minList.append(y[2])
    dfForImpute[column+'_mean'] = meanList
    dfForImpute[column+'_max'] = maxList
    dfForImpute[column+'_min'] = minList

dfForImpute = dfForImpute[[i for i in dfForImpute.columns if i != 0]]

In [ ]:
# add on non-time data for imputation
dfForImpute = dfForImpute.copy()
extraColumns = [i for i in dfDroppedMissing.columns if i not in list(dfForImpute.columns) + timeseries + ['infection', 'vent_array', 'int_time1', 'ext_time1'] + ptinfo + adm_num]
print(extraColumns)
for i in extraColumns:
    if i in ('weight', 'height', 'duration1'):
        dfForImpute[i] = df[i]
    else:
        dfForImpute[i] = df[i].astype('category')


In [ ]:
# before imputation again
dfForImpute

In [ ]:
dfForImpute2 = dfForImpute[dfForImpute.columns[:]]

kds = mf.ImputationKernel(
  dfForImpute2,
  datasets=1,
  save_all_iterations=True,
  random_state=1991
)

# Run the MICE algorithm for 3 iterations
kds.mice(3)

print(kds)

dfImputed = kds.complete_data(dataset=0, inplace=False)
print(dfImputed.isnull().sum(0))

# after imputation
dfImputed.head(10)

In [ ]:
y = np.asarray([i.to_pydatetime() for i in df["int_time1"]])
z = np.asarray([i.to_pydatetime() for i in df["outtime"]])
dfImputed['icu_stay_duration'] = [i.total_seconds() for i in z-y]
dfImputed[['hadm_id','subject_id']] = df[['hadm_id','subject_id']]
dfImputed

In [ ]:
# adding on critical rows that were removed when yeeting columns with high missing values

for i in ('dod', 'deathtime'):
    dfImputed[i] = df[i]

In [ ]:
dfImputed.to_csv('imputed.csv')

## 2.0: Exploring language and ventilation duration 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score

dfImputed_onehot = dfImputed.copy()
dfImputed1 = pd.get_dummies(dfImputed_onehot, columns = ['ethnicity', 'language'], prefix = ['eth', 'lang'])
onehot_cols = [i for i in list(dfImputed1.columns) if i not in ['ethnicity', 'marital_status', 'language', 'admission_location']]
dfImputed1['duration1'] = df['duration1']
dfImputed1 = dfImputed1.dropna(axis=0, how='any')

In [ ]:
dfImputed1

In [ ]:
from matplotlib import pyplot
from scipy.stats import kstest, norm, pearsonr, ttest_ind

x = dfImputed1['lang_ENGL']
y = df['duration1'].dropna(axis=0,how='any')

#testing for normalcy
mu, std = norm.fit(x)
n = norm(loc=mu, scale=std)
normalcy_p = kstest(x, n.cdf)[1]
print('p-val: '+ str(normalcy_p))

pyplot.scatter(x,y)
covariance = np.cov(x,y)
print('Covariance matrix: ')
print(covariance)

#find pearson R
corr, _ = pearsonr(x,y)
print('Pearsons R: %.3f' %corr)

In [ ]:
import statsmodels.api as sm

univariable = pd.DataFrame(index = ['eth_white','lang_ENGL'], columns = ['OR (95% CI)','p-value'])

p_value = []
OR = []
time = []
for column in ['lang_ENGL']:
    for int_timelimit in range(0,48,1):
        X = dfImputed1[column].values
        X = sm.add_constant(X)
        y = pd.cut(df['duration1'].dropna(how='any'),bins=[-0.1,int_timelimit,4000],labels=[0,1])
        logit = sm.Logit(y, X)
        model = logit.fit(method='bfgs',disp = False)
        model_odds = pd.DataFrame(np.exp(model.params), columns=['OR'])
        model_odds['p-value'] = model.pvalues
        model_odds[['2.5%','97.5%']]= np.exp(model.conf_int())
        model_odds.rename(index={'x1': column}, inplace = True)
        # insert it into the dataframe 'univariable'
        univariable.at[column,'OR (95% CI)'] = '{OR:.3f} ({lower:.3f} to {upper:.3f})'.format(OR = model_odds.at[column, 'OR'], 
                                                                                            lower = model_odds.at[column, '2.5%'], 
                                                                                            upper = model_odds.at[column, '97.5%'])
        univariable.at[column,'p-value'] = '{0:.3f}'.format(model_odds.at[column, 'p-value'])
        p_value.append(model_odds.at[column, 'p-value'])
        OR.append(model_odds.at[column, 'OR'])
        time.append(int_timelimit)

univariable

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(time, OR, label='OR',color='blue')
ax.set_xlabel("ventilation time")
ax.set_ylabel("OR")

ax2=ax.twinx()
ax2.plot(time, p_value, label='p-val',color='orange')
ax2.set_ylabel("p-val")


In [ ]:
x1 = dfImputed1['duration1'][dfImputed1['lang_ENGL']==1].dropna(axis=0,how='any')
x2 = dfImputed1['duration1'][dfImputed1['lang_ENGL']==0].dropna(axis=0,how='any')
plt.hist(x1,bins=[-0.1,2,4,6,10,13,16,20,25,30,40,50,60],range=(0,60))
plt.hist(x2,bins=[-0.1,2,4,6,10,13,16,20,25,30,40,50,60],range=(0,60),color='orange')
print('mean English: '+str(np.mean(x1)))
print('mean non-English: '+str(np.mean(x2)))
print('t_test p-val;: '+str(ttest_ind(x1, x2)[1]))

## 2.1: DL Survival Analysis

In [ ]:
new_cols = ['Unnamed: 0','hadm_id','subject_id','gender','ethnicity','marital_status','insurance','language','aortic','mit','tricuspid',
            'pulmonary','cabg','temp','bg_temp','hr','spo2','rr','sbp','dbp','meanbp','weight','height','cardiac_index','pt','ptt',
            'inr','inr_1','fibrinogen','hb','hematocrit','plts','wcc','lymphocytes','neutrophils','alp','ast','alt','ggt',
            'bilirubin_indirect','bilirubin_direct','bilirubin_total','chloride','magnesium','potassium','crp','bleed_time','albumin',
            'creatinine','free_calcium','sodium','bicarb','bun','hba1c','glucose','lactate','po2','pco2','baseexcess','ph','aado2',
            'fio2','ffp','insulin','cryo','prbc','infection','ventrate','tidalvol','vent_array','reintubation','liver_severe','liver_mild',
            'rheum','cvd','aids','ckd','copd','arrhythmia','pud','smoking','pvd','paraplegia','ccf','met_ca','t2dm','t1dm','malig','mi',
            'dementia','first_careunit','last_careunit','admission_location','admission_type','hospital_expire_flag','admittime',
            'dischtime','intime','outtime','ext_time','reint_time','los','icustay_seq','deathtime','plt','diab_un','diab_cc',
            'dtoutput','specimen','dod']

# variables for analysis
timeSeriesMMM=[]
for column in timeseries:
    for a in ['_mean','_max','_min']:
        timeSeriesMMM.append(column+a)
input_cols = ['gender','ethnicity','marital_status','insurance','language','aortic','mit','tricuspid',
            'pulmonary','cabg','weight','height','liver_severe','liver_mild','rheum','cvd','aids','ckd','copd','arrhythmia',
            'pud','smoking','pvd','paraplegia','ccf','met_ca','t2dm','t1dm','malig','mi','dementia','first_careunit',
            'admission_location','admission_type','diab_un','diab_cc',*timeSeriesMMM]
outcome_cols = ['reintubation','hospital_expire_flag','los','duration1','duration2']
others = ['admittime','dischtime','intime','outtime','ext_time','reint_time','deathtime','icustay_seq','dtoutput','specimen','dod',
        'int_time1','ext_time1','int_time2','ext_time2']


In [ ]:
templist=[]
for column in timeseries:
    for a in ['_mean','_max','_min']:
        templist.append(column+a)
print(templist)